# Capstone Project of Summer Analytics 2025
 - hosted by Consulting & Analytics Club × Pathway

# Inroduction


####  Urban parking spaces are a limited and highly demanded resource. Prices that remain static
 throughout the day can lead to inefficiencies — either overcrowding or underutilization.
 To improve utilization, dynamic pricing based on demand, competition, and real-time
 conditions is crucial.


In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
df = pd.read_csv('dataset.csv')
df


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [ ]:
import pandas as pd

# Example: df has a column 'SystemCodeNumber' with 14 unique values
grouped_datasets = {group: data for group, data in df.groupby('SystemCodeNumber')}


In [ ]:
df['SystemCodeNumber'].unique()

array(['BHMBCCMKT01', 'BHMBCCTHL01', 'BHMEURBRD01', 'BHMMBMMBX01',
       'BHMNCPHST01', 'BHMNCPNST01', 'Broad Street', 'Others-CCCPS105a',
       'Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202',
       'Others-CCCPS8', 'Others-CCCPS98', 'Shopping'], dtype=object)

# Make the dataset for a slot from above 14 slots for which you want to predict the price

In [ ]:
df= grouped_datasets['BHMBCCMKT01'] # Only data where SystemCodeNumber == 'BHMBCCMKT01'
df


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1307,1307,BHMBCCMKT01,577,26.144536,91.736172,309,car,average,4,0,19-12-2016,14:30:00
1308,1308,BHMBCCMKT01,577,26.144536,91.736172,300,cycle,low,3,0,19-12-2016,15:03:00
1309,1309,BHMBCCMKT01,577,26.144536,91.736172,274,cycle,low,2,0,19-12-2016,15:29:00
1310,1310,BHMBCCMKT01,577,26.144536,91.736172,230,car,average,2,0,19-12-2016,16:03:00


In [ ]:
df['TrafficConditionNearby'].unique()

array(['low', 'high', 'average'], dtype=object)

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

In [ ]:
df.columns = df.columns.str.strip()  # Remove tab characters

df_clean = df[["Timestamp", "SystemCodeNumber", "Occupancy", "Capacity",
               "QueueLength", "TrafficConditionNearby", "IsSpecialDay",
               "VehicleType", "Latitude", "Longitude"]]

df_clean.to_csv("parking_stream.csv", index=False)

In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Price_Model 1

In [ ]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime
base_price=10
alpha=5

delta_window1 = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        # Pricing Formula:
        #    price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #  the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6 '''

        price=base_price+ alpha* (pw.this.occ_max - pw.this.occ_min) / pw.this.cap,
    )
)


# Price_Model 2

In [ ]:
base_price = 10
alpha, beta, gamma, delta = 0.5, 0.3, 0.2, 0.4
lambda_ = 0.8
delta_window2= (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),   # Maximum capacity observed (typically constant per spot)
        IsSpecialDay=pw.reducers.max(pw.this.IsSpecialDay),
        queue=pw.reducers.max(pw.this.QueueLength),
        traffic = {"low": 0.6, "average": 1.0, "high": 1.4}.get(pw.this.TrafficConditionNearby, 1.0),
        vehicle_weight = {"car": 1.0, "bike": 0.6, "truck": 1.4}.get(pw.this.VehicleType, 1.0),
    )
    .with_columns(
        #occupancy_rate =(pw.this.occ_max - pw.this.occ_min) / pw.this.cap,
        #special_day = pw.this.IsSpecialDay,
        #traffic = {"low": 0.6, "average": 1.0, "high": 1.4}.get(pw.this.TrafficConditionNearby, 1.0),
        #vehicle_weight = {"car": 1.0, "bike": 0.6, "truck": 1.4}.get(pw.this.VehicleType, 1.0),
        price = base_price * (1 + lambda_ * (alpha *(pw.this.occ_max - pw.this.occ_min) / pw.this.cap +
              beta * pw.this.queue -
              gamma * 0 +
              delta * pw.this.IsSpecialDay +
              pw.this.vehicle_weight)/5)
    )
)

# Price_Model 3

In [ ]:
'''from geopy.distance import geodesic

# Dummy competitor prices and locations (replace with real stream if needed)
competitor_prices = {
    "P1": 12, "P2": 14, "P3": 9, "P4": 11
}
lot_locations = {
    "P1": (28.6129, 77.2295),
    "P2": (28.6130, 77.2294),
    "P3": (28.6125, 77.2300),
    "P4": (28.6131, 77.2298)
}

def model_3_pricing(lot_id, timestamp, lat, lon, capacity, occupancy,
                    queue, vehicle_type, traffic, is_special_day):
    demand_price = model_2_pricing(lot_id, timestamp, lat, lon, capacity, occupancy,
                                   queue, vehicle_type, traffic, is_special_day)

    current_location = (lat, lon)
    nearby_prices = []
    for other_lot, loc in lot_locations.items():
        if other_lot == lot_id:
            continue
        dist = geodesic(current_location, loc).meters
        if dist <= 300:
            nearby_prices.append(competitor_prices.get(other_lot, 10))

    if nearby_prices:
        avg_nearby = np.mean(nearby_prices)
        if occupancy >= capacity and demand_price > avg_nearby:
            demand_price -= 2  # Encourage rerouting
        elif demand_price < avg_nearby:
            demand_price += 1  # Increase price slightly

    return np.clip(demand_price, 5, 20)'''

In [ ]:
'''delta_window3= (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(

        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)'''

#  Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

#### use delta_window1/2/3 respectively for price model 1/2 or 3.  

In [ ]:
# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window1.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()